### Heroes Of Pymoli Data Analysis
 - Above 80% of all Heros of Pymoli players identify as male, but spend the least per user referencing normalized totals 
 - On average, no gender category purchases more items/person than another
 - 55% of all players are between the ages of 18-26

In [1]:
import pandas as pd
import numpy as np

In [2]:
data_file = "purchase_data.json"
data = pd.read_json(data_file,orient = 'columns')
#data.columns

** Player Count 

In [3]:
uc = len(data['SN'].unique())
usercount = pd.DataFrame({'Total Players':uc},index=[0])
usercount

,Total Players
0,573


** Purchasing Analysis (Total)

In [4]:
# Number of Unique Items 
itemcount = len(data['Item Name'].unique())

# Average Purchase Price 
avgprice = data['Price'].mean()

# Total Number of Purchases
purchasecnt = len(data['Price'])

# Total Revenue sum of purchases
totalRevenue = data['Price'].sum()

#push into DF
purchAnalysis = pd.DataFrame({'Number of Unique Items':itemcount,'Average Price':avgprice,
                              'Number of Purchases':purchasecnt,'Total Revenue':totalRevenue},index=[0])
#format currency
purchAnalysis['Average Price'] = purchAnalysis['Average Price'].map("${0:,.2f}".format)
purchAnalysis['Total Revenue'] = purchAnalysis['Total Revenue'].map("${0:,.2f}".format)

#format for correct order and new object format
purchAnalysis = purchAnalysis[['Number of Unique Items','Average Price','Number of Purchases','Total Revenue']]
purchAnalysis

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$2.93,780,"$2,286.33"


** Gender Demographics

In [5]:
# unique SN grouped by gender
new_data = data[['Gender', 'SN']].drop_duplicates().groupby(['Gender'])
#gender counts
gen_count = new_data['Gender'].count()
#gender percents
gen_perc = gen_count/gen_count.sum()*100
unique_gender = pd.DataFrame({"Percentage of Players":gen_perc,
                            "Total Count": gen_count})
unique_gender["Percentage of Players"] = unique_gender["Percentage of Players"].map("{0:,.2f}%".format)
# # unique_gender = unique_gender.set_index('Gender')

unique_gender

,Percentage of Players,Total Count
Gender,,
Female,17.45%,100
Male,81.15%,465
Other / Non-Disclosed,1.40%,8


** Purchasing Analysis (Gender)

In [6]:
gender_group = data.groupby(['Gender'])
gender_cnt = gender_group['Price'].count()
gender_avg = gender_group['Price'].mean()
gender_sum = gender_group['Price'].sum()
gender_norm = gender_sum/gen_count

gender_purchase = pd.DataFrame({"Purchase Count": gender_cnt,
                                "Average Purchase Price": gender_avg,
                                "Total Purchase Value":gender_sum,
                                "Normalized Totals": gender_norm
                                })

#Format
gender_purchase ['Average Purchase Price'] = gender_purchase ['Average Purchase Price'].map("${0:,.2f}".format)
gender_purchase ['Total Purchase Value'] = gender_purchase ['Total Purchase Value'].map("${0:,.2f}".format)
gender_purchase ['Normalized Totals'] = gender_purchase ['Normalized Totals'].map("${0:,.2f}".format)
#Reorder columns
gender_purchase = gender_purchase[['Purchase Count','Average Purchase Price','Total Purchase Value','Normalized Totals']] 
#Set Index
# gender_purchase = gender_purchase.set_index('Gender')
gender_purchase 


,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Gender,,,,
Female,136,$2.82,$382.91,$3.83
Male,633,$2.95,"$1,867.68",$4.02
Other / Non-Disclosed,11,$3.25,$35.74,$4.47


**Age Demographics

In [7]:
# Unique Users
uniqueuser=pd.DataFrame(data.groupby('SN')['Gender'].max())
uniqueuser.reset_index(inplace=True)

# Age broken into bins of 4 years (i.e. <10, 10-14, 15-19, etc.) 
maxage = data["Age"].max()
# determine how many bins will be needed
i = ((maxage/4).round())-1

# generate bins 
bins = [age*4+10 for age in range(int(i))] 

#change tuple into list to append 0 to the front of the list and cast back into a tuple
bins = list(bins)
bins.insert(0,0)
bins = tuple(bins)

# must use loop to create dynamic naming for different datasets
group_names = []
for item in range(len(bins)):
    group_names.append(str(bins[item-1])+' - '+ str(bins[item]))
#remove group name that called -1
del group_names[0]

uniqueuser["Age Range"] = pd.cut(data["Age"], bins, labels=group_names)
# cut for later
data["Age Range"] = pd.cut(data["Age"], bins, labels=group_names)

In [8]:
age_data = pd.DataFrame(uniqueuser.groupby('Age Range')['Gender'].count())
percent = age_data['Gender']/age_data['Gender'].sum()*100
age_data["Percentage of Players"] = percent.map("{0:,.2f}%".format)
age_data = age_data.rename(columns={'Gender': 'Total Count'})
age_data = age_data[['Percentage of Players','Total Count']]
age_data


,Percentage of Players,Total Count
Age Range,,
0 - 10,3.84%,22
10 - 14,4.01%,23
14 - 18,13.79%,79
18 - 22,28.80%,165
22 - 26,27.92%,160
26 - 30,8.38%,48
30 - 34,5.93%,34
34 - 38,4.19%,24
38 - 42,2.97%,17


** Purchasing Analysis Age

In [9]:
age_total = data.groupby(['Age Range'])
age_sum = age_total['Price'].sum()
age_avg = age_total['Price'].mean()
age_count = age_total['Price'].count()
age_norm = age_sum/age_data['Total Count']

age_purchase = pd.DataFrame({"Total Purchase Value":age_sum,"Average Purchase Price":age_avg,"Purchase Count":age_count,"Normalized Totals":age_norm})
age_purchase = age_purchase[["Purchase Count","Average Purchase Price","Total Purchase Value","Normalized Totals"]]
age_purchase["Average Purchase Price"] = age_purchase["Average Purchase Price"].map("${0:,.2f}".format)
age_purchase["Total Purchase Value"] = age_purchase["Total Purchase Value"].map("${0:,.2f}".format)
age_purchase["Normalized Totals"] = age_purchase["Normalized Totals"].map("${0:,.2f}".format)

age_purchase

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Age Range,,,,
0 - 10,32,$3.02,$96.62,$4.39
10 - 14,31,$2.70,$83.79,$3.64
14 - 18,111,$2.88,$319.32,$4.04
18 - 22,231,$2.93,$676.20,$4.10
22 - 26,207,$2.94,$608.02,$3.80
26 - 30,63,$2.98,$187.99,$3.92
30 - 34,46,$3.07,$141.24,$4.15
34 - 38,37,$2.81,$104.06,$4.34
38 - 42,20,$3.13,$62.56,$3.68


** Top Spenders

In [10]:
# Identify the the top 5 spenders in the game by total purchase value, then list (in a table):
spenders = data.groupby(['SN'])

spend_count = spenders['Item Name'].count()
spend_average = spenders['Price'].mean()
spend_total = spenders['Price'].sum()

top_spenders = pd.DataFrame({"Total Purchase Value":spend_total,
                             "Average Purchase Price":spend_average,
                             "Purchase Count":spend_count})
top_spenders = top_spenders[["Purchase Count","Average Purchase Price","Total Purchase Value"]]
top_spenders["Average Purchase Price"] = top_spenders["Average Purchase Price"].map("${0:,.2f}".format)
top_spenders["Total Purchase Value"] = top_spenders["Total Purchase Value"].map("${0:,.2f}".format)
top_spenders = top_spenders.sort_values("Total Purchase Value", ascending = False) 
top_spenders.head(5)


,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Qarwen67,4,$2.49,$9.97
Sondim43,3,$3.13,$9.38
Tillyrin30,3,$3.06,$9.19
Lisistaya47,3,$3.06,$9.19
Tyisriphos58,2,$4.59,$9.18


** Most Popular Items                

In [11]:
items = data.groupby(['Item ID','Item Name'])
item_count = items['Price'].count()
item_max = items['Price'].max() 
item_sum = items['Price'].sum()

item_pop = pd. DataFrame({"Purchase Count":item_count,"Item Price":item_max,"Total Purchase Value": item_sum})
item_pop["Item Price"] = item_pop["Item Price"].map("${0:,.2f}".format)
item_pop["Total Purchase Value"] = item_pop["Total Purchase Value"].map("${0:,.2f}".format)
item_pop = item_pop [["Purchase Count","Item Price","Total Purchase Value"]]
item_pop = item_pop.sort_values("Purchase Count", ascending = False)
item_pop.head(5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
39,"Betrayal, Whisper of Grieving Widows",11,$2.35,$25.85
84,Arcane Gem,11,$2.23,$24.53
31,Trickster,9,$2.07,$18.63
175,Woeful Adamantite Claymore,9,$1.24,$11.16
13,Serenity,9,$1.49,$13.41


** Most Profitable Items

In [12]:
#  I think I am not examining this deeply enough
item_prof = item_pop.sort_values("Total Purchase Value", ascending = False)
item_pop.head(5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
39,"Betrayal, Whisper of Grieving Widows",11,$2.35,$25.85
84,Arcane Gem,11,$2.23,$24.53
31,Trickster,9,$2.07,$18.63
175,Woeful Adamantite Claymore,9,$1.24,$11.16
13,Serenity,9,$1.49,$13.41
